In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Wed May 28 09:58:51 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   60C    P8             10W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
%%capture
!pip install datasets transformers huggingface_hub torchaudio librosa jiwer
!pip install accelerate -U
!pip install nvidia-smi
!pip install transformers[torch]
!pip install wandb
!pip install matplotlib seaborn
!pip install evaluate
import os
import re
import wandb
import torch
import numpy as np
import json
from evaluate import load
from datasets import load_dataset, Audio
from transformers import EarlyStoppingCallback
from huggingface_hub import HfFolder


In [ ]:
from datasets import load_dataset, DatasetDict

test5hr = load_dataset("KYAGABA/amharic_cleaned_testset_verified")['val']
print(test5hr)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Resolving data files:   0%|          | 0/23 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/23 [00:00<?, ?it/s]

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/31954 [00:00<?, ? examples/s]

Generating val split:   0%|          | 0/2553 [00:00<?, ? examples/s]

NotImplementedError: Loading a dataset cached in a LocalFileSystem is not supported.

In [ ]:
import unicodedata
import re
def preprocess_text(batch):
    batch['transcription'] = [text.lower() for text in batch['transcription']]
    batch['transcription'] = [unicodedata.normalize('NFKC', text) for text in batch['transcription']]
    batch['transcription'] = [re.sub(r'[\’\ʻ\ʼ\ʽ\‘\']', '', text) for text in batch['transcription']]
    batch['transcription'] = [re.sub(r'[^\w\s\']', '', text) for text in batch['transcription']]
    batch['transcription'] = [re.sub(r'_', ' ', text) for text in batch['transcription']]
    batch['transcription'] = [' '.join(text.split()) for text in batch['transcription']]
    return batch

test5hr = test5hr.map(preprocess_text, batched=True)

In [ ]:
def extract_all_chars(batch):
    all_text = " ".join(batch["transcription"])
    vocab = list(set(all_text))
    return {"vocab": [vocab], "all_text": [all_text]}
vocab_test = test5hr.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=test5hr.column_names)
vocab_list = list(set(vocab_test["vocab"][0]))
vocab_dict = {v: k for k, v in enumerate(sorted(vocab_list))}
vocab_dict

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available else 'cpu')
device

In [ ]:
import evaluate

wer_metric = evaluate.load("wer")
cer_metric = evaluate.load("cer")

In [ ]:
from transformers import AutoProcessor, pipeline

model_name = "asr-africa/facebook-mms-1b-all-common_voice_fleurs-amh-200hrs-v1"

# Load the processor/tokenizer explicitly
processor = AutoProcessor.from_pretrained(model_name)

# Pass the processor to the pipeline
pipe = pipeline("automatic-speech-recognition", model=model_name, tokenizer=processor.tokenizer, device=device)

In [ ]:
from transformers import AutoTokenizer

model_name = "asr-africa/facebook-mms-1b-all-common_voice_fleurs-amh-200hrs-v1"

# Try loading the tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)
print(tokenizer.vocab_size)  # Check if the vocabulary is properly loaded

269


In [ ]:
from transformers.pipelines.pt_utils import KeyDataset
from transformers import pipeline
from tqdm import tqdm

pipe = pipeline("automatic-speech-recognition",model= "asr-africa/facebook-mms-1b-all-common_voice_fleurs-amh-200hrs-v1", device =device)

In [ ]:
def transcribe(audio):
    text = pipe(audio)["text"]
    return text

In [ ]:
predictions = [prediction['text'] for prediction in tqdm(pipe(KeyDataset(test5hr, "audio"), batch_size=32),\
                                                              desc= "Running inference", total= test5hr.num_rows)]

In [ ]:
test5hr = test5hr.add_column("predictions", predictions)

In [ ]:
print("Test WER: {:.4f}".format(wer_metric.compute(predictions=test5hr["predictions"], references=test5hr["transcription"])))
print("Test CER: {:.4f}".format(cer_metric.compute(predictions=test5hr["predictions"], references=test5hr["transcription"])))

In [ ]:
from datasets import load_dataset, DatasetDict
import torch
from transformers.pipelines.pt_utils import KeyDataset
from transformers import pipeline
from tqdm import tqdm
import evaluate
import pandas as pd

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

models = [

    "asr-africa/whisper-small-amharic_dataset-ormo-100hrs-v6",

]

wer_metric = evaluate.load("wer")
cer_metric = evaluate.load("cer")

def compute_individual_metrics(prediction, reference):
    wer = wer_metric.compute(predictions=[prediction], references=[reference])
    cer = cer_metric.compute(predictions=[prediction], references=[reference])
    return wer, cer

def run_inference_for_model(model_name):
    print(f"Processing model: {model_name}")
    pipe = pipeline("automatic-speech-recognition", model=model_name, device=device, return_timestamps='word')
    results = []
    all_predictions = []
    all_references = []

    for audio, transcription in tqdm(zip(test5hr["audio"], test5hr["transcription"]),
                                                         total=test5hr.num_rows, desc="Running inference"):
        prediction = pipe(audio)['text']
        wer, cer = compute_individual_metrics(prediction, transcription)

        results.append({
            "Prediction": prediction,
            "Reference": transcription,
            "WER": wer,
            "CER": cer,
            # "Source": source
        })
        all_predictions.append(prediction)
        all_references.append(transcription)

    df = pd.DataFrame(results)
    overall_wer = wer_metric.compute(predictions=all_predictions, references=all_references)
    overall_cer = cer_metric.compute(predictions=all_predictions, references=all_references)

    print("Calculated overall WER:", overall_wer)
    print("Calculated overall CER:", overall_cer)
    # average_metrics_by_source = df.groupby("Source").agg({"WER": "mean", "CER": "mean"}).reset_index()
    # print("\nAverage WER and CER by Source:")
    # print(average_metrics_by_source)
    # Save detailed results to CSV
    output_filename = f'transcriptions_{model_name.split("/")[-1]}.csv'
    # output_filename = f"{model_name}_detailed_results.csv"
    df.to_csv(output_filename, index=False)
    print(f"Results saved to {output_filename}")

# Loop through each model and evaluate
for model in models:
    run_inference_for_model(model)

Processing model: asr-africa/whisper-small-amharic_dataset-ormo-100hrs-v6


config.json:   0%|          | 0.00/1.38k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/967M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/3.92k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/283k [00:00<?, ?B/s]

...

added_tokens.json:   0%|          | 0.00/34.6k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.19k [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/339 [00:00<?, ?B/s]

Running inference:   0%|          | 0/2553 [00:00<?, ?it/s]/usr/local/lib/python3.11/dist-packages/transformers/models/whisper/generation_whisper.py:573: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(
Running inference:  68%|██████▊   | 1728/2553 [2:17:32<1:05:39,  4.78s/it]


IndexError: string index out of range

In [ ]:
!pip install jiwer

  Using cached jiwer-3.1.0-py3-none-any.whl.metadata (2.6 kB)
  Using cached rapidfuzz-3.12.2-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (12 kB)
Using cached jiwer-3.1.0-py3-none-any.whl (22 kB)
Using cached rapidfuzz-3.12.2-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (3.1 MB)


In [ ]:
import os
import json
import re
import torch
import librosa
import numpy as np
import pandas as pd
from datasets import load_dataset, Audio, concatenate_datasets
from transformers import WhisperProcessor, WhisperForConditionalGeneration
import evaluate
from huggingface_hub import HfApi
from tqdm import tqdm

# Initialize WER and CER metrics
wer_metric = evaluate.load("wer")
cer_metric = evaluate.load("cer")

# List of models to evaluate
models = [
    "asr-africa/whisper-small-amharic_dataset-ormo-100hrs-v6",

]
# Device configuration
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")
# Define Audio feature with consistent sampling rate
audio_feature = Audio(sampling_rate=16000, mono=True)
# Define data preprocessing and helper functions
def unify_and_add_source(batch, dataset_name):
    # Look for different text column names and standardize
    if 'transcription' in batch:
        text = batch['transcription']
    elif 'sentence' in batch:
        text = batch['sentence']
    elif 'text' in batch:
        text = batch['text']
    elif 'transcript' in batch:
        text = batch['transcript']
    else:
        text = ""

    batch['sentence'] = text
    batch['dataset_source'] = dataset_name
    return batch

def preprocess_text(batch):
    batch['sentence'] = re.sub(r'[^\w\s\'፡።፣፤፥፦፨]', '', batch['sentence'].lower()).strip()
    return batch

def get_lens(batch):
    try:
        audio_len = librosa.get_duration(y=batch['audio']['array'], sr=batch['audio']['sampling_rate'])
    except:
        audio_len = 0.0
    transcript_len = len(batch['sentence'])
    batch.update({
        'audio_len': audio_len,
        'transcript_len': transcript_len,
        'len_ratio': audio_len / transcript_len if transcript_len > 0 else 0,
        'num_feature_vecs': int(np.round(audio_len * 1000 / 20))
    })
    return batch

def data_checks(batch):
    audio_check = 1.0 < batch['audio_len'] < 30.0
    transcript_check = batch['transcript_len'] > 10
    input_output_ratio_check = (batch['num_feature_vecs'] / batch['transcript_len']) > 1.0 if batch['transcript_len'] > 0 else False
    return audio_check and transcript_check and input_output_ratio_check

def select_audio_by_hours(dataset, target_duration=4500):
    cumulative_duration, selected_indices = 0.0, []
    for idx, audio_data in enumerate(dataset["audio"]):
        duration = librosa.get_duration(y=audio_data["array"], sr=audio_data["sampling_rate"])
        cumulative_duration += duration
        if cumulative_duration > target_duration:
            break
        selected_indices.append(idx)
    return dataset.select(selected_indices)

# Dataset details with split configuration
datasets_info = {
    "AMHARIC_ALFA": {"path": "KYAGABA/AMHARIC_ALFA_DATASET_new_current", "split": "train"},
    "FLEURS": {"path": "KYAGABA/fleurs_voice_luo_new", "split": "train"},
    "COMMON_VOICE": {"path": "KYAGABA/amharic_cleaned_testset_common_voice_new", "split": "test"},
    "AMHARIC_DATASET": {"path": "KYAGABA/amharic_cleaned_testset_verified", "split": "val"}
}

# Load and preprocess datasets
def load_and_preprocess_datasets(datasets_info, target_duration=500):
    processed_datasets = {}

    for dataset_name, info in datasets_info.items():
        print(f"\nLoading dataset: {dataset_name}")
        dataset_path, split = info["path"], info["split"]

        try:
            dataset = load_dataset(dataset_path, split=split).cast_column("audio", audio_feature)
            original_size = len(dataset)

            # Apply preprocessing
            dataset = dataset.map(lambda batch: unify_and_add_source(batch, dataset_name))
            dataset = dataset.map(preprocess_text)
            dataset = dataset.map(get_lens)
            dataset = dataset.filter(data_checks)

            # Select limited duration for efficiency
            dataset = select_audio_by_hours(dataset, target_duration=target_duration)

            filtered_size = len(dataset)
            print(f"Dataset {dataset_name}: Original size = {original_size}, Filtered size = {filtered_size}")

            processed_datasets[dataset_name] = dataset

        except Exception as e:
            print(f"Error processing dataset {dataset_name}: {str(e)}")
            continue

    return processed_datasets

# Run inference for a specific model on all datasets
def evaluate_model(model_name, processed_datasets):
    print(f"\n{'='*20} Evaluating model: {model_name} {'='*20}")

    try:
        # Load model and processor
        processor = WhisperProcessor.from_pretrained(model_name)
        model = WhisperForConditionalGeneration.from_pretrained(model_name)
        model.to(device)
        model.eval()

        dataset_results = {}
        all_predictions = []
        all_references = []

        # Process each dataset
        for dataset_name, dataset in processed_datasets.items():
            print(f"\nRunning inference on {dataset_name} dataset")
            predictions, references = [], []

            # Prepare features only once for this model
            print("Preparing input features...")
            dataset = dataset.map(
                lambda batch: {
                    "input_features": processor(
                        batch["audio"]["array"],
                        sampling_rate=batch["audio"]["sampling_rate"],
                        return_tensors="pt"
                    ).input_features.squeeze().numpy()
                },
                desc="Processing audio",
                num_proc=4
            )

            # Run inference
            for batch in tqdm(dataset, desc=f"Inferencing {dataset_name}"):
                input_features = torch.tensor(batch["input_features"]).unsqueeze(0).to(device)

                with torch.no_grad():
                    generated_ids = model.generate(input_features)

                transcription = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]
                predictions.append(transcription)
                references.append(batch["sentence"])

            # Calculate metrics
            wer = wer_metric.compute(predictions=predictions, references=references)
            cer = cer_metric.compute(predictions=predictions, references=references)

            print(f"Results for {dataset_name} - WER: {wer}, CER: {cer}")

            # Save individual results
            dataset_results[dataset_name] = {
                "WER": wer,
                "CER": cer,
                "Predictions": predictions,
                "References": references
            }

            all_predictions.extend(predictions)
            all_references.extend(references)

        # Calculate overall metrics
        overall_wer = wer_metric.compute(predictions=all_predictions, references=all_references)
        overall_cer = cer_metric.compute(predictions=all_predictions, references=all_references)

        print(f"\nOverall results for {model_name} - WER: {overall_wer}, CER: {overall_cer}")

        # Save detailed results
        save_results(model_name, dataset_results, overall_wer, overall_cer)

        return {
            "Model": model_name,
            "Overall_WER": overall_wer,
            "Overall_CER": overall_cer,
            "Dataset_Results": dataset_results
        }

    except Exception as e:
        print(f"Error evaluating model {model_name}: {str(e)}")
        return {
            "Model": model_name,
            "Error": str(e)
        }

# Save results to CSV files
def save_results(model_name, dataset_results, overall_wer, overall_cer):
    model_short_name = model_name.split("/")[-1]

    # Create directory for results if it doesn't exist
    os.makedirs("results", exist_ok=True)

    # Save detailed transcription results for each dataset
    for dataset_name, results in dataset_results.items():
        df_detailed = pd.DataFrame({
            "Reference": results["References"],
            "Prediction": results["Predictions"]
        })

        # Add WER and CER for each example
        df_detailed["Individual_WER"] = [
            wer_metric.compute(predictions=[pred], references=[ref])
            for pred, ref in zip(results["Predictions"], results["References"])
        ]

        df_detailed["Individual_CER"] = [
            cer_metric.compute(predictions=[pred], references=[ref])
            for pred, ref in zip(results["Predictions"], results["References"])
        ]

        # Save dataset-specific results
        output_file = f"results/{model_short_name}_{dataset_name}_results.csv"
        df_detailed.to_csv(output_file, index=False)
        print(f"Saved detailed results to {output_file}")

    # Save summary for this model
    summary_rows = []
    for dataset_name, results in dataset_results.items():
        summary_rows.append({
            "Model": model_short_name,
            "Dataset": dataset_name,
            "WER": results["WER"],
            "CER": results["CER"],
            "Samples": len(results["Predictions"])
        })

    # Add overall row
    summary_rows.append({
        "Model": model_short_name,
        "Dataset": "OVERALL",
        "WER": overall_wer,
        "CER": overall_cer,
        "Samples": sum(len(results["Predictions"]) for results in dataset_results.values())
    })

    summary_file = f"results/{model_short_name}_summary.csv"
    pd.DataFrame(summary_rows).to_csv(summary_file, index=False)
    print(f"Saved summary results to {summary_file}")

# Main execution
if __name__ == "__main__":
    # Set maximum duration per dataset (in seconds)
    MAX_DURATION_PER_DATASET = 900  # 15 minutes per dataset for faster evaluation

    # Load and preprocess all datasets
    print("\nLoading and preprocessing datasets...")
    processed_datasets = load_and_preprocess_datasets(datasets_info, target_duration=MAX_DURATION_PER_DATASET)

    # Store overall results
    overall_results = []

    # Evaluate each model
    for model_name in models:
        result = evaluate_model(model_name, processed_datasets)
        overall_results.append(result)

    # Create comparison of all models
    if overall_results:
        comparison_rows = []
        for result in overall_results:
            if "Error" not in result:
                model_short_name = result["Model"].split("/")[-1]

                # Overall metrics
                comparison_rows.append({
                    "Model": model_short_name,
                    "Dataset": "OVERALL",
                    "WER": result["Overall_WER"],
                    "CER": result["Overall_CER"]
                })

                # Per-dataset metrics
                for dataset_name, dataset_result in result["Dataset_Results"].items():
                    comparison_rows.append({
                        "Model": model_short_name,
                        "Dataset": dataset_name,
                        "WER": dataset_result["WER"],
                        "CER": dataset_result["CER"]
                    })

        # Save comparison
        comparison_file = "results/model_comparison.csv"
        pd.DataFrame(comparison_rows).to_csv(comparison_file, index=False)
        print(f"\nSaved model comparison to {comparison_file}")

    print("\nEvaluation complete!")

In [ ]:
import unicodedata
import re

def preprocess_text(batch):
    # Convert to lowercase
    batch['text'] = batch['text'].lower()

    # First normalize to NFD to separate base characters and diacritics
    batch['text'] = unicodedata.normalize('NFD', batch['text'])

    # Remove diacritical marks (accents)
    batch['text'] = re.sub(r'[\u0300-\u036f]', '', batch['text'])

    # Only keep basic Latin alphabet (a-z) and spaces
    batch['text'] = re.sub(r'[^a-z\s]', '', batch['text'])

    # Remove excessive whitespace
    batch['text'] = ' '.join(batch['text'].split())

    return batch

# Example usage to demonstrate:
test_text = "Hélló Wörld! Àfrîca's Oromo language - it's béautiful №42."
result = preprocess_text({"text": test_text})
print(f"Original: {test_text}")
print(f"Processed: {result['text']}")
# Output should be: "hello world africas oromo language its beautiful"

# Resulting vocabulary will only contain: a b c d e f g h i j k l m n o p q r s t u v w x y z space

Original: Hélló Wörld! Àfrîca's Oromo language - it's béautiful №42.
Processed: hello world africas oromo language its beautiful
